In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"

In [2]:
_QUERY="""
select * from patient;
"""
QUERY="""
SELECT
    p.patient_code,
    tmi.*
FROM
    tracking_motherbasicinfo tmi
LEFT JOIN
    patient p 
ON
    p.id = tmi.id_patient;
"""

MUSO_WITH_COMPTAGEMENAGE = """
SELECT *
FROM
    caris_db.view_muso_household_report
where (!isnull(indice_householdcount) or (is_household_applicable ='yes' ))
"""

In [3]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

True

In [4]:
dataset = read_sql_query(MUSO_WITH_COMPTAGEMENAGE,engine,parse_dates=True)

In [5]:
engine.dispose()

In [6]:
""" INFO="mother_tracking_info"
_INFO="patient"
dataset.to_excel(f"../../pvvih_sansComptage/{_INFO}.xlsx",index=False) """

' INFO="mother_tracking_info"\n_INFO="patient"\ndataset.to_excel(f"../../pvvih_sansComptage/{_INFO}.xlsx",index=False) '

In [8]:
dataset.to_excel('../../muso_with_CM.xlsx',index=False)